In [6]:
import numpy as np

def CC_DailyFlow(BT_MonthlyFlow, HydroCond_Ann, Precip):
    
    # Future years
    WY = np.loadtxt('./Input/WY.txt')

    # Historical BT daily flows
    BTDFlows = np.loadtxt('./Input/BT_Historical_DFlows.txt')

    # Convert monthly rainfall to inches per month
    Precip[:, 3] = Precip[:, 2] / 10 / 2.54
    Precip = np.c_[Precip, np.zeros((len(Precip),2))] # add two extra columns to precip, will add values later

    # Get the length for use below
    HydroLength = len(BT_MonthlyFlow)

    Monthly_Projection = np.zeros((HydroLength, 8))

    for j in range(HydroLength):
        
        a = BT_MonthlyFlow[j, 2]
        b = BT_MonthlyFlow[j, 1]

        result = str(int(a)) + str(int(b))
        result2 = float(result)

        Monthly_Projection[j, 0] = result2

    # Build the monthly projected flow matrix with passed data
    Monthly_Projection[:, 1] = BT_MonthlyFlow[:, 2]
    Monthly_Projection[:, 2] = BT_MonthlyFlow[:, 1]
    Monthly_Projection[:, 3] = np.ceil(BT_MonthlyFlow[:, 4])
    Monthly_Projection[:, 4] = np.ceil(BT_MonthlyFlow[:, 5])
    # Build the hydrologic condition matrix with passed data
    HC_Hgraph = np.ceil(HydroCond_Ann)

    ##########
    # LOAD UP THE HYDROLOGIC AND RAINFALL DATA FOR USE
    # IN DECOMPOSING THE MONTHLY FLOWS INTO DAILY FLOWS
    CDry_Hydrograph = pd.read_csv('./Input/CC_CritDryHydrograph.txt', delimiter = '\t', header = None, skiprows = [12,25,38]).values
    Dry_Hydrograph = pd.read_csv('./Input/CC_DryHydrograph.txt', delimiter = '\t', header = None, skiprows = [12,25,38]).values
    Avg_Hydrograph = pd.read_csv('./Input/CC_AvgHydrograph.txt', delimiter = '\t', header = None, skiprows = [12,25,38]).values
    Wet_Hydrograph = pd.read_csv('./Input/CC_WetHydrograph.txt', delimiter = '\t', header = None, skiprows = [12,25,38]).values
    ExtWet_Hydrograph = pd.read_csv('./Input/CC_ExtWetHydrograph.txt', delimiter = '\t', header = None, skiprows = [12,25,38]).values
    # Here are the monthly rainfalls to go with the annual hydrographs
    CDry_Precip = np.loadtxt('./Input/CC_CritDryRainfall.txt', delimiter = '\t')
    Dry_Precip = np.loadtxt('./Input/CC_DryRainfall.txt', delimiter = '\t')
    Avg_Precip = np.loadtxt('./Input/CC_AvgRainfall.txt', delimiter = '\t')
    Wet_Precip = np.loadtxt('./Input/CC_WetRainfall.txt', delimiter = '\t')
    ExtWet_Precip = np.loadtxt('./Input/CC_ExtWetRainfall.txt', delimiter = '\t')
    # Need to sum and normalize the rainfall data for testing below
    CDry_Precip_CD = np.cumsum(CDry_Precip, axis = 0)
    Dry_Precip_CD = np.cumsum(Dry_Precip, axis = 0)
    Avg_Precip_CD = np.cumsum(Avg_Precip, axis = 0)
    Wet_Precip_CD = np.cumsum(Wet_Precip, axis = 0)
    ExtWet_Precip_CD = np.cumsum(ExtWet_Precip, axis = 0)
    # Now normalize the cumsum precip to the largest value
    CDry_Precip_CDN = CDry_Precip_CD / np.max(CDry_Precip_CD, axis = 0)
    Dry_Precip_CDN = Dry_Precip_CD / np.max(Dry_Precip_CD, axis = 0)
    Avg_Precip_CDN = Avg_Precip_CD / np.max(Avg_Precip_CD, axis = 0)
    Wet_Precip_CDN = Wet_Precip_CD / np.max(Wet_Precip_CD, axis = 0)
    ExtWet_Precip_CDN = ExtWet_Precip_CD / np.max(ExtWet_Precip_CD, axis = 0)
    ##########

    dateLength = int(np.sum(BT_MonthlyFlow[:, 0]))
    CC_ProjTimeSeries = np.zeros((dateLength, 12))

    # Date at start of time series
    A = np.datetime64(str(int(BT_MonthlyFlow[0, 2])) + '-' + str(int(BT_MonthlyFlow[0, 1])) + '-01')
    B = np.datetime64(str(int(BT_MonthlyFlow[-1, 2])) + '-0' + str(int(BT_MonthlyFlow[-1, 1])) + '-30')
    ymd = pd.date_range(start = A, end = B)
    y = np.array([val.year for val in ymd])
    m = np.array([val.month for val in ymd])
    d = np.array([val.day for val in ymd])
    CC_ProjTimeSeries[:, 0] = m #ymd.astype('datetime64[M]').astype(int) % 12 + 1
    CC_ProjTimeSeries[:, 1] = d #ymd.astype('datetime64[D]').astype(int) % 30 + 1
    CC_ProjTimeSeries[:, 2] = y #ymd.astype('datetime64[Y]').astype(int) + 1970

    for j in range(dateLength):
        
        b1 = str(int(CC_ProjTimeSeries[j, 2]))
        b2 = str(int(CC_ProjTimeSeries[j, 0]))
        str_val = b1 + b2
        CC_ProjTimeSeries[j, 3] = int(str_val)

    # This reshapes the hydrograph matrices into column-wise arrangement
    # There are four columns and each column is a different water year
    # hydrograph used for decomposing the monthly flows depending on
    # the hydrologic conditions
    # CDry_HydrographArray = CDry_Hydrograph.T.flatten().reshape(-1, 4)
    # Dry_HydrographArray = Dry_Hydrograph.T.flatten().reshape(-1, 4)
    # Avg_HydrographArray = Avg_Hydrograph.T.flatten().reshape(-1, 4)
    # Wet_HydrographArray = Wet_Hydrograph.T.flatten().reshape(-1, 4)
    # ExtWet_HydrographArray = ExtWet_Hydrograph.T.flatten().reshape(-1, 4)

    CDry_HydrographArray = CDry_Hydrograph.flatten().reshape(4,-1).T
    Dry_HydrographArray = Dry_Hydrograph.flatten().reshape(4,-1).T
    Avg_HydrographArray = Avg_Hydrograph.flatten().reshape(4,-1).T
    Wet_HydrographArray = Wet_Hydrograph.flatten().reshape(4,-1).T
    ExtWet_HydrographArray = ExtWet_Hydrograph.flatten().reshape(4,-1).T

    m2, _ = Monthly_Projection.shape
    m3, _ = HC_Hgraph.shape

    # Lets grab the monthly precip values for the Precip matrix so we can
    # use it to figure out the best distribution match to identify the annual
    # hydrograph for use in decomposing the monthly flows
    NoYears = len(Precip) // 12
    # AnnPrecip_Mat = Precip[:, 3].reshape(-1, NoYears)  # 51
    AnnPrecip_Mat = Precip[:, 3].reshape(NoYears,-1).T  # 51

    AnnPrecip_Mat_CD = np.cumsum(AnnPrecip_Mat, axis=0)
    
    # Now normalize the cumsum precip to the largest value
    AnnPrecip_Mat_CDN = AnnPrecip_Mat_CD / np.max(AnnPrecip_Mat_CD, axis = 0)

    # Need a temp variable to store the results of comparing distributions
    Dist_Compare = np.zeros((m3, 4))

    # This builds the monthly matrices for decomposing done below
    for j in range(m3):

        if HC_Hgraph[j, 1] == 1:

            for jj in range(4):

                Dist = np.cumsum(np.abs(AnnPrecip_Mat_CDN[:, j] - CDry_Precip_CDN[:, jj]))
                
                Dist_Compare[j, jj] = Dist[-1]

                if jj == 3:

                    minimum = np.min(np.min(Dist_Compare[j, :]))
                    y = np.where(Dist_Compare[j, :] == minimum)[0] + 1

                    if len(y) > 1:

                        r = np.random.randint(1, y[1])
                        winner = r
                        HC_Hgraph[j, 3] = winner

                    else:

                        # This is index to use to select the annual flow record
                        # in the identified hydrologic condition matrix
                        HC_Hgraph[j, 3] = y[0]

        elif HC_Hgraph[j, 1] == 2:

            for jj in range(4):

                Dist = np.cumsum(np.abs(AnnPrecip_Mat_CDN[:, j] - Dry_Precip_CDN[:, jj]))

                Dist_Compare[j, jj] = Dist[-1]

                if jj == 3:

                    minimum = np.min(np.min(Dist_Compare[j, :]))
                    y = np.where(Dist_Compare[j, :] == minimum)[0] + 1

                    if len(y) > 1:

                        r = np.random.randint(1, y[1])
                        winner = r
                        HC_Hgraph[j, 3] = winner

                    else:

                        # This is index to use to select the annual flow record
                        # in the identified hydrologic condition matrix
                        HC_Hgraph[j, 3] = y[0]

        elif HC_Hgraph[j, 1] == 3:

            for jj in range(4):

                Dist = np.cumsum(np.abs(AnnPrecip_Mat_CDN[:, j] - Avg_Precip_CDN[:, jj]))

                Dist_Compare[j, jj] = Dist[-1]

                if jj == 3:

                    minimum = np.min(np.min(Dist_Compare[j, :]))
                    y = np.where(Dist_Compare[j, :] == minimum)[0] + 1

                    if len(y) > 1:

                        r = np.random.randint(1, y[1])
                        winner = r
                        HC_Hgraph[j, 3] = winner

                    else:

                        # This is index to use to select the annual flow record
                        # in the identified hydrologic condition matrix
                        HC_Hgraph[j, 3] = y[0]

        elif HC_Hgraph[j, 1] == 4:

            for jj in range(4):

                Dist = np.cumsum(np.abs(AnnPrecip_Mat_CDN[:, j] - Wet_Precip_CDN[:, jj]))

                Dist_Compare[j, jj] = Dist[-1]

                if jj == 3:

                    minimum = np.min(np.min(Dist_Compare[j, :]))
                    y = np.where(Dist_Compare[j, :] == minimum)[0] + 1

                    if len(y) > 1:

                        r = np.random.randint(1, y[1])
                        winner = r
                        HC_Hgraph[j, 3] = winner

                    else:

                        # This is index to use to select the annual flow record
                        # in the identified hydrologic condition matrix
                        HC_Hgraph[j, 3] = y[0]

        elif HC_Hgraph[j, 1] == 5:

            for jj in range(4):

                Dist = np.cumsum(np.abs(AnnPrecip_Mat_CDN[:, j] - ExtWet_Precip_CDN[:, jj]))

                Dist_Compare[j, jj] = Dist[-1]

                if jj == 3:

                    minimum = np.min(np.min(Dist_Compare[j, :]))
                    y = np.where(Dist_Compare[j, :] == minimum)[0] + 1

                    if len(y) > 1:

                        r = np.random.randint(1, y[1])
                        winner = r
                        HC_Hgraph[j, 3] = winner

                    else:

                        # This is index to use to select the annual flow record
                        # in the identified hydrologic condition matrix
                        HC_Hgraph[j, 3] = y[0]
    
    # This builds the daily matrices to store the decomposed flow
    for j in range(m2):

        for jj in range(m3):

            if Monthly_Projection[j,1] == HC_Hgraph[jj,0] - 1 and Monthly_Projection[j,2] > 9:

                Monthly_Projection[j, 5] = HC_Hgraph[jj, 1]
                Monthly_Projection[j, 6] = HC_Hgraph[jj, 2]
                # This is the index to sample the annual hydrograph in each
                # water year based on the preceding loop operations
                Monthly_Projection[j, 7] = HC_Hgraph[jj, 3]
                Precip[j, 4] = HC_Hgraph[jj, 1]

            elif Monthly_Projection[j,1] == HC_Hgraph[jj,0] and Monthly_Projection[j,2] < 10:

                Monthly_Projection[j, 5] = HC_Hgraph[jj, 1]
                Monthly_Projection[j, 6] = HC_Hgraph[jj, 2]
                # This is the index to sample the annual hydrograph in each
                # water year based on the preceding loop operations
                Monthly_Projection[j, 7] = HC_Hgraph[jj, 3]
                Precip[j, 4] = HC_Hgraph[jj, 1]

            elif Monthly_Projection[j, 1] == 2070 and Monthly_Projection[j, 2] > 9:

                Monthly_Projection[j, 5] = HC_Hgraph[jj, 1]
                Monthly_Projection[j, 6] = HC_Hgraph[jj, 2]
                # This is the index to sample the annual hydrograph in each
                # water year based on the preceding loop operations
                Monthly_Projection[j, 7] = HC_Hgraph[jj, 3]
                Precip[j, 4] = HC_Hgraph[jj, 1]

            if Monthly_Projection[j, 2] == 9:

                Precip[j, 5] = np.sum(Precip[j - 11:j+1, 3])
                # Precip[j - 11:j - 1, 5] = Precip[j, 5]
                Precip[j - 11:j, 5] = Precip[j, 5]

            elif Monthly_Projection[j, 1] == 2070 and Monthly_Projection[j, 2] > 9:

                # The value of 44.6 was sourced from cal-adpat for all 10
                # models and is the average annual total of 2070 over the san
                # lorenzo watershed
                Precip[j, 5] = 44.6

    cum_month = 9
    # Add water year total precip to the Ann Hydro condition matrix
    for j in range(len(HydroCond_Ann)):

        yr = HydroCond_Ann[j, 0]

        for jj in range(len(Precip)):

            yr_check = Precip[jj, 0]
            month_check = Precip[jj, 1]

            ##### Commented out b/c it is never used
            # if yr_check == yr and month_check == cum_month: 
            #     HydroCond_Ann[j, 3] = Precip[jj, 5]
  
    m, _ = CC_ProjTimeSeries.shape

    # This builds the daily matrices to store the decomposed flow
    for j in range(m):

        for jj in range(m2):

            if CC_ProjTimeSeries[j,2] == Monthly_Projection[jj,1] and CC_ProjTimeSeries[j,0] == Monthly_Projection[jj,2]:
                
                CC_ProjTimeSeries[j,4] = Monthly_Projection[jj,5]
                CC_ProjTimeSeries[j,5] = Monthly_Projection[jj,6]
                CC_ProjTimeSeries[j,11] = Monthly_Projection[jj,7]
                CC_ProjTimeSeries[j,9] = Precip[jj,5]
            
    m1, _ = CDry_HydrographArray.shape

    # Decompose the monthly flow record into a daily flow record year by year,
    # based on the hydrologic condition and the selection of a year for
    # decomposing the record within each hydrologic condition. The variable idx
    # is a pointer to the annual hydrograph with the closest monthly distribution of rainfall
    #
    # This is the decay rate constant. At this point this is a guess...
    lambda_val = 250
    counter = 1

    for j in range(m):

        for jj in range(m1):

            if CC_ProjTimeSeries[j, 0] == WY[jj, 0] and CC_ProjTimeSeries[j, 1] == WY[jj, 1]:

                if CC_ProjTimeSeries[j, 4] == 1:

                    # If statement to deal with water years with no
                    # precipitation
                    if CC_ProjTimeSeries[j, 9] == 0 and j > 372:

                        init_value = CC_ProjTimeSeries[j - counter, 6]
                        decay_rate = np.exp(-(1 / lambda_val) * jj)
                        CC_ProjTimeSeries[j, 6] = init_value * decay_rate
                        CC_ProjTimeSeries[j, 10] = 0

                        nan_check = np.isnan(WY[jj, 0])

                        if nan_check == 0:

                            counter = counter + 1

                    else:

                        idx = int(CC_ProjTimeSeries[j, 11]) - 1
                        CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5] * CDry_HydrographArray[jj, idx]
                        CC_ProjTimeSeries[j, 10] = CC_ProjTimeSeries[j, 9] * CDry_HydrographArray[jj, idx]

                elif CC_ProjTimeSeries[j, 4] == 2:

                    idx = int(CC_ProjTimeSeries[j, 11]) - 1
                    CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5] * Dry_HydrographArray[jj, idx]
                    CC_ProjTimeSeries[j, 10] = CC_ProjTimeSeries[j, 9] * Dry_HydrographArray[jj, idx]

                elif CC_ProjTimeSeries[j, 4] == 3:

                    idx = int(CC_ProjTimeSeries[j, 11]) - 1
                    CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5] * Avg_HydrographArray[jj, idx]
                    CC_ProjTimeSeries[j, 10] = CC_ProjTimeSeries[j, 9] * Avg_HydrographArray[jj, idx]

                elif CC_ProjTimeSeries[j, 4] == 4:

                    idx = int(CC_ProjTimeSeries[j, 11]) - 1
                    CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5] * Wet_HydrographArray[jj, idx]
                    CC_ProjTimeSeries[j, 10] = CC_ProjTimeSeries[j, 9] * Wet_HydrographArray[jj, idx]

                elif CC_ProjTimeSeries[j, 4] == 5:

                    idx = int(CC_ProjTimeSeries[j, 11]) - 1
                    CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5] * ExtWet_HydrographArray[jj, idx]
                    CC_ProjTimeSeries[j, 10] = CC_ProjTimeSeries[j, 9] * ExtWet_HydrographArray[jj, idx]

        if CC_ProjTimeSeries[j, 0] == 9 and CC_ProjTimeSeries[j, 1] == 30:

            counter = 1

    # Correct the flow record at the end of each water year to provide a 
    # smooth transition for year to year. Otherwise abrupt changes can occur.
    # Define a backward looking vector length to spread abrupt flow changes
    # This is for the first case < 1.15; i.e. decrease flow at WY
    back_1 = 30
    # This is for the second case > 1.15; i.e. increase in flow at WY
    back_2 = 122
    forward = 122
    efold_back = 1/15
    efold_forward = 1/500
    peak = 25

    for j in range(m):

        if j < m-1:

            if CC_ProjTimeSeries[j,0] == 9 and CC_ProjTimeSeries[j,1] == 30:

                CC_ProjTimeSeries[j,7] = CC_ProjTimeSeries[j,6] / CC_ProjTimeSeries[j+1,6]

                if (CC_ProjTimeSeries[j,6] / CC_ProjTimeSeries[j+1,6]) > 1.15:
                    
                    anchor_back = CC_ProjTimeSeries[j+1-back_1,6]
                    delta1 = CC_ProjTimeSeries[j+1-back_1,6]-CC_ProjTimeSeries[j+1,6]
                    array_back = np.arange(back_1,0,-1)
                                                    
                    for jj in np.arange(back_1,0,-1, dtype = 'int'):
                        
                        CC_ProjTimeSeries[j+1-jj,8] = anchor_back - (delta1 - (delta1 * np.exp(-efold_back*array_back[jj-1])))
                    
                if (CC_ProjTimeSeries[j+1,6] / CC_ProjTimeSeries[j,6]) > 1.15:

                    idx = np.where(CC_ProjTimeSeries[j:(j+forward)+1,6] > peak)[0][0] # potentially -1
                    
                    if np.size(idx) > 0:
                        
                        forward_find = j + (idx-2)
                                                   
                        for jj in np.arange(j,forward_find+2):
                        
                            if jj == j + 1:
                                
                                CC_ProjTimeSeries[jj,8] = CC_ProjTimeSeries[jj-1,6] - (efold_forward * CC_ProjTimeSeries[jj-1,6])                      
                        
                            else:
                                
                                CC_ProjTimeSeries[jj,8] = CC_ProjTimeSeries[jj-1,8] - (efold_forward * CC_ProjTimeSeries[jj-1,8])
                        
                    else:
                        
                        anchor_back = CC_ProjTimeSeries[(j+1)-back_2,6]
                        delta1 = CC_ProjTimeSeries[(j+1)-back_2,6]-CC_ProjTimeSeries[(j+1),6]
                        array_back = np.arange(back_2,0,-1)
                                                    
                        for jj in np.arange(back_2,0,-1, dtype = 'int'):
                        
                            CC_ProjTimeSeries[(j+1)-jj,8] = anchor_back - (delta1 - (delta1 * np.exp(-efold_back*array_back[jj-1])))
           
            else:
            
                CC_ProjTimeSeries[j,7] = 0

    for j in range(m):

        if CC_ProjTimeSeries[j, 8] == 0:

            CC_ProjTimeSeries[j, 8] = CC_ProjTimeSeries[j, 6]

        if CC_ProjTimeSeries[j, 0] == 2 and CC_ProjTimeSeries[j, 1] == 29:

            CC_ProjTimeSeries[j, 8] = (CC_ProjTimeSeries[j - 1, 6] + CC_ProjTimeSeries[j + 1, 6]) / 2

    FinalFlow = np.zeros((m, 5))
    FinalFlow[:, 0] = CC_ProjTimeSeries[:, 0]
    FinalFlow[:, 1] = CC_ProjTimeSeries[:, 1]
    FinalFlow[:, 2] = CC_ProjTimeSeries[:, 2]
    FinalFlow[:, 3] = CC_ProjTimeSeries[:, 8]
    FinalFlow[:, 4] = CC_ProjTimeSeries[:, 10]

    BT = FinalFlow

    return BT, HC_Hgraph